## This notebook demonstrates the use of a simple meal planner expert system using Canada's Food and Nutrition database as the Knowledge Base

In [1]:
#import libraries
import numpy as np
import pandas as pd
import random
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.display import display

In [2]:
food_data_df = pd.read_csv('FoodNutritionData.csv')
display(food_data_df.head(5))

,FoodID,FoodDescription,FoodGroup,PROTValue,FATValue,CARBValue,STARValue,TSUGValue,TDFValue,TSATValue,MUFAValue,PUFAValue
0,2,Cheese souffle,Mixed Dishes,9.54,15.70,5.91,0.00,2.66,0.1,5.742,5.820,2.770
1,4,"Chop suey, with meat, canned",Mixed Dishes,4.07,2.80,5.29,0.00,3.40,1.1,0.364,1.540,0.750
2,5,"Chinese dish, chow mein, chicken",Mixed Dishes,6.76,2.80,8.29,3.99,1.74,1.0,0.490,0.613,1.226
3,6,Corn fritter,Baked Products,8.55,21.24,38.62,0.00,2.85,2.0,5.455,8.543,5.564
4,7,"Beef pot roast, with browned potatoes, peas an...",Mixed Dishes,21.29,5.25,10.72,0.00,1.44,1.6,1.872,2.552,0.709


In [3]:
def get_ingredient_data_row(ingredient_input,food_df):
    ingredient_name = ingredient_input[0]
    all_food_upper = food_df.FoodDescription.tolist()
    all_food = [food.lower().replace(",", " ") for food in all_food_upper]
    matched_food = process.extractOne(ingredient_name,all_food,scorer=fuzz.token_set_ratio)
    #matched_foods = [food[0] for food in matched_foods]
    #matched_food = process.extractOne(ingredient_name,matched_foods,scorer=fuzz.partial_ratio)
    matched_food = all_food_upper[all_food.index(matched_food[0])]
    temp_df = food_df[food_df.FoodDescription == matched_food].copy()
    return {
        "FoodName":matched_food,
        "FoodGroup":temp_df.FoodGroup.tolist()[0],
        "TotalAmount":ingredient_input[1],
        "PROTValue":temp_df.PROTValue.tolist()[0],
        "FATValue":temp_df.FATValue.tolist()[0],
        "CARBValue":temp_df.CARBValue.tolist()[0],
        "STARValue":temp_df.STARValue.tolist()[0],
        "TSUGValue":temp_df.TSUGValue.tolist()[0],
        "TDFValue":temp_df.TDFValue.tolist()[0],
        "TSATValue":temp_df.TSATValue.tolist()[0],
        "MUFAValue":temp_df.MUFAValue.tolist()[0],
        "PUFAValue":temp_df.PUFAValue.tolist()[0]
        }

def get_ingredient_df(ingredient_inp_list,food_df):
    ingredient_list = []
    for ing in ingredient_inp_list:
        ingredient_list.append(get_ingredient_data_row(ing,food_df))
    return pd.DataFrame(ingredient_list)

In [4]:
class UserData():
    
    def __init__(self):
        """Initialize User Data Variables"""
        self.age = None
        self.height = None
        self.gender = None
        self.weight = None
        self.prot_req = None
        self.carb_req = None
        self.fat_req = None
        self.ingredient_df = None
        
    def input_user_data(self,input_list):
        """Use input list to initialize user data"""
        self.age = input_list[0]
        self.height = input_list[1]
        self.gender = input_list[2]
        self.weight = input_list[3]                
        
    def calculate_daily_food_requirements(self):
        """This function uses the Harris-Benedict Equation for Basal Energy Expenditure to 
           to calculate the daily requirement of protein, fat and carbohydrates"""
        if self.gender == 1:
            multipliers = [655.1,9.6,1.9,4.7]
        else:
            multipliers = [66.5,13.8,5.0,6.8]
        total_calories = multipliers[0] + multipliers[1] * self.weight\
                        + multipliers[2] * self.height + multipliers[3] * self.age
        self.prot_req = self.weight
        prot_per = ((self.weight * 4)/total_calories) * 100
        carb_per = 60
        fat_per = 100 - (prot_per + carb_per)
        self.carb_req = ((total_calories * carb_per)/100)/4
        self.fat_req = ((total_calories * fat_per)/100)/9
        print("You require",self.prot_req,"g of protein",self.carb_req,"g of carbohydrates",\
             self.fat_req,"g of fats per day")

In [5]:
user = UserData()
user.input_user_data([25,178,0,95]) #[Age,Height,Gender(0 for male 1 for female),weight]
user.calculate_daily_food_requirements()

You require 95 g of protein 365.625 g of carbohydrates 66.11111111111111 g of fats per day


In [6]:
ingredient_list = [
    ["bread whole wheat homemade",1000.0],
    ["bread white commercial toasted",2000.0],
    ["beef brain raw",1000.0],
    ["chicken broiler breast raw",3000.0],
    ["Cereal ready to eat Fibre First Multibran",500.0],
    ["Milk fluid skim",500.0],
    ["cheese cheddar",1000.0],
    ["fish salmon atlantic wild raw",2000.0],
    ["mexican burrito with beans",1500.0],
    ["spinach raw",2000.0],
    ["apple red delicious raw",3000.0],
    ["tuna salad",1000.0],
    ["sweet potato raw",2500.0],
    ["banana raw",1000.0],
    ["beans kidney dark red",1200.0]
]

In [7]:
user.ingredient_df = get_ingredient_df(ingredient_list,food_data_df)
display(user.ingredient_df.head(15))

,FoodName,FoodGroup,TotalAmount,PROTValue,FATValue,CARBValue,STARValue,TSUGValue,TDFValue,TSATValue,MUFAValue,PUFAValue
0,"Bread, whole wheat, homemade (2/3 whole wheat ...",Baked Products,1000.0,8.40,5.40,51.40,0.00,3.84,6.0,0.796,1.158,2.939
1,"Bread, white with raisins, commercial, toasted",Baked Products,2000.0,8.15,4.40,60.84,0.00,19.76,4.8,1.272,0.905,2.027
2,"Beef, brain, raw",Beef Products,1000.0,10.86,10.30,1.05,0.00,0.00,0.0,2.300,1.890,1.586
3,"Chicken, broiler, breast, meat and skin, raw",Poultry Products,3000.0,20.85,9.25,0.00,0.00,0.00,0.0,2.660,3.820,1.960
4,"Cereal, ready to eat, Fibre First Multibran, B...",Breakfast cereals,500.0,11.20,3.70,78.20,0.00,18.00,43.0,0.700,0.800,2.000
5,"Milk, fluid, skim",Dairy and Egg Products,500.0,3.37,0.08,4.96,0.00,5.09,0.0,0.056,0.022,0.003
6,"Cheese, processed, cheddar, cold pack",Dairy and Egg Products,1000.0,19.66,24.46,8.32,0.00,0.51,0.0,15.355,7.165,0.719
7,"Fish, salmon, atlantic, wild, raw",Finfish and Shellfish Products,2000.0,19.84,6.34,0.00,0.00,0.00,0.0,0.981,2.103,2.539
8,"Fast foods, mexican, burrito with beans",Fast Foods,1500.0,6.48,6.22,32.92,0.00,0.00,4.4,3.174,2.184,0.551
9,"Mustard spinach (tendergreen), raw",Vegetables and Vegetable Products,2000.0,2.20,0.30,3.90,0.00,0.00,2.8,0.015,0.138,0.057


In [19]:
class Meal():
    
    def __init__(self,ingredients=None,random_state=None,no_of_ingredients=None,override_ingredients=None):
        """Creates a Random Meal from User Data OR from specified dataframe"""
        self.fitness = 0.0
        if override_ingredients is not None:
            self.ingredients = override_ingredients
            return
        
        ingredient_df = ingredients.sample(n = no_of_ingredients,random_state = random_state)
        ingred_vals = []
        for i,row in ingredient_df.iterrows():
            values = {
                "FoodName":row.FoodName,
                "FoodGroup":row.FoodGroup,
                "TotalAmount":row.TotalAmount,
                "MealAmount":float(random.random() * row.TotalAmount),
                "PROTValue":row.PROTValue,
                "FATValue":row.FATValue,
                "CARBValue":row.CARBValue,
                "STARValue":row.STARValue,
                "TSUGValue":row.TSUGValue,
                "TDFValue":row.TDFValue,
                "TSATValue":row.TSATValue,
                "MUFAValue":row.MUFAValue,
                "PUFAValue":row.PUFAValue
            }
            ingred_vals.append(values)
        self.ingredients = pd.DataFrame(ingred_vals)
        
    def update_fitness(self,prot_req,carb_req,fat_req):
        total_vals = {
            "PROTValue":0.0,
            "FATValue":0.0,
            "CARBValue":0.0,
            "STARValue":0.0,
            "TSUGValue":0.0,
            "TDFValue":0.0,
            "TSATValue":0.0,
            "MUFAValue":0.0,
            "PUFAValue":0.0
        }
        for i,row in self.ingredients.iterrows():
            for column in self.ingredients.columns:
                if column in total_vals:
                    total_vals[column] = total_vals[column] + (row.MealAmount * row[column])/100
        #apply rules to calculate fitness
        #for protein
        #add two if protein needs met
        if total_vals["PROTValue"] > prot_req - 1 and total_vals["PROTValue"] < prot_req + 1:
            self.fitness = self.fitness + 2 
        #subtract one if protein needs not met
        elif total_vals["PROTValue"] < prot_req:
            self.fitness = self.fitness - 1
        #subtract two if protein exceeds since that is worse
        elif total_vals["PROTValue"] > prot_req:
            self.fitness = self.fitness - 2
        #for carbohydrate
        #do same as protein for total carb value
        if total_vals["CARBValue"] > carb_req - 1 and total_vals["CARBValue"] < carb_req + 1:
            self.fitness = self.fitness + 2
        elif total_vals["CARBValue"] < carb_req:
            self.fitness = self.fitness - 1
        elif total_vals["CARBValue"] > carb_req:
            self.fitness = self.fitness - 2
        # subtract 1*(TSUG/TotalCarb)            
        self.fitness = self.fitness - (total_vals["TSUGValue"]/total_vals["CARBValue"])
        # add 1*(STAR/TotalCarb)        
        self.fitness = self.fitness + (total_vals["STARValue"]/total_vals["CARBValue"])
        # add 2*(TDF/TotalCarb)        
        self.fitness = self.fitness + 2 * (total_vals["TDFValue"]/total_vals["CARBValue"])
        #for fat
        #do same as protein for total fat value
        if total_vals["FATValue"] > carb_req - 1 and total_vals["FATValue"] < carb_req + 1:
            self.fitness = self.fitness + 2
        elif total_vals["FATValue"] < carb_req:
            self.fitness = self.fitness - 1
        elif total_vals["FATValue"] > carb_req:
            self.fitness = self.fitness - 2        
        #subtract 1*(TSAT/TotalFat)
        self.fitness = self.fitness - (total_vals["TSATValue"]/total_vals["FATValue"])        
        #add 1*(MUFA/TotalFat)
        self.fitness = self.fitness + (total_vals["MUFAValue"]/total_vals["FATValue"])        
        #add 1*(PUFA/TotalFat)
        self.fitness = self.fitness + 2 * (total_vals["PUFAValue"]/total_vals["FATValue"])        
        

class DailyMealPlanner():
    
    def __init__(self,
                 user_data,
                 combinations = 100,
                 no_of_ingredients = 5,
                 mutation_rate=0.001,
                 elite_percentage=10,
                 random_state=1):
        """Initialize The Meal Plan System"""
        self.combinations = combinations
        self.no_of_ingredients = no_of_ingredients
        self.mutation_rate = mutation_rate
        self.elite_per_unit = elite_percentage/100.0
        self.random_state = random_state
        self.user_data = user_data
    
    def create_population(self):
        """Get Random Samples from ingredient data"""
        random.seed(self.random_state)
        self.current_generation = []
        for i in range(0,self.combinations):
            self.current_generation.append(Meal(ingredients=self.user_data.ingredient_df,
                                                random_state=self.random_state,
                                                no_of_ingredients=self.no_of_ingredients))
            
    def update_generation_order(self):
        """Updates the fitness function of the meal and sets the order of the current generation"""
        for meal in self.current_generation:
            meal.update_fitness(self.user_data.prot_req,self.user_data.carb_req,self.user_data.fat_req)
        self.current_generation = sorted(self.current_generation, key=lambda meal: meal.fitness, reverse=True)
        
    def perform_selection(self):
        """Performs the selection step of genetic algorithm"""
        total_selected = int(self.elite_per_unit * len(self.current_generation))
        selected_generations = []
        for i in range(0,total_selected):
            selected_generations.append(self.current_generation[i])
        # randomly pick candidates from the rest
        selected_generations.extend(random.sample(self.current_generation[int(self.elite_per_unit):], 10))
        self.current_generation = selected_generations
    
    def perform_ordered_cross_over(self):
        """Performs the ordered cross over step of genetic algorithm"""
        self.current_generation = random.sample(self.current_generation,len(self.current_generation))
        children = []
        while len(children) < self.combinations:
            x,y = random.sample(range(len(self.current_generation)), 2)
            parent_meal_1 = self.current_generation[x]
            parent_meal_2 = self.current_generation[y]
            parent_1_genes = parent_meal_1.ingredients.sample(n=random.randint(1,self.no_of_ingredients - 1),
                                                              random_state = self.random_state)
            parent_2_genes = parent_meal_2.ingredients.sample(n=self.no_of_ingredients - len(parent_1_genes),
                                                             random_state = self.random_state)
            child = Meal(override_ingredients = parent_1_genes.append(parent_2_genes,ignore_index=True))
            children.append(child)
        self.current_generation = children
        
    def perform_mutation(self):
        """Performs the mutation step of genetic algorithm"""
        mutated = []
        for meal in self.current_generation:
            ingredients = meal.ingredients.to_dict(orient='records')
            for i in range(self.no_of_ingredients):
                if (random.random() < self.mutation_rate):
                    j = int(random.random() * self.no_of_ingredients)
                    val_i = ingredients[i]['MealAmount']
                    val_j = ingredients[j]['MealAmount']
                    ingredients[i]['MealAmount'] = max(min(val_j,ingredients[i]['TotalAmount']),0)
                    ingredients[j]['MealAmount'] = max(min(val_i,ingredients[j]['TotalAmount']),0)
            meal.ingredients = pd.DataFrame(ingredients)
        
    def next_generation(self):
        """Runs a single iteration of genetic algorithm"""
        self.update_generation_order()
        self.perform_selection()
        self.perform_ordered_cross_over()
        self.perform_mutation()
            
    def get_healthy_meal(self,generations = 100):
        """Runs the Genetic Algorithm to get a new meal"""
        self.create_population()
        for i in range(0,generations):
            self.next_generation()
        return self.current_generation[0]

In [20]:
daily_meal_planner = DailyMealPlanner(user,200,5,0.002,15,25)
new_meal = daily_meal_planner.get_healthy_meal(500)
print("Meal Fitness:" ,new_meal.fitness)
display(new_meal.ingredients.head(5))

Meal Fitness: 0.0


,FoodName,FoodGroup,TotalAmount,MealAmount,PROTValue,FATValue,CARBValue,STARValue,TSUGValue,TDFValue,TSATValue,MUFAValue,PUFAValue
0,"Banana, raw",Fruits and fruit juices,1000.0,15.783814,1.09,0.33,22.84,5.38,12.23,1.7,0.112,0.032,0.073
1,"Banana, raw",Fruits and fruit juices,1000.0,15.783814,1.09,0.33,22.84,5.38,12.23,1.7,0.112,0.032,0.073
2,"Banana, raw",Fruits and fruit juices,1000.0,15.783814,1.09,0.33,22.84,5.38,12.23,1.7,0.112,0.032,0.073
3,"Banana, raw",Fruits and fruit juices,1000.0,15.783814,1.09,0.33,22.84,5.38,12.23,1.7,0.112,0.032,0.073
4,"Banana, raw",Fruits and fruit juices,1000.0,15.783814,1.09,0.33,22.84,5.38,12.23,1.7,0.112,0.032,0.073
